In [515]:
from datetime import datetime, timedelta
from functools import reduce
import time
import urllib, http.client
from urllib.parse import urlparse, urlencode
import json
import csv
import os
import sys
import unittest

In [510]:
class Poloniex():
    "Класс для запроса данных с биржи Poloniex"
    
    def __init__(self, currency_pair, period):
        """
        :param currency_pair: пара валют в виде BTC_ETH
        :param period: период, за который строятся свечи (в секундах)
        """
        self.currency_pair = currency_pair
        self.period = period
    
    def __request_bars(self, start, end):
        """
        Запрос на биржу Poloniex
        :param start: начало периода (timestamp)
        :param end: конец периода (timestamp)
        """
        params = {
            'method': 'public',
            'command': 'returnChartData',
            'currencyPair': self.currency_pair,
            'start': start,
            'end': end,
            'period': self.period,
        }
        url = 'https://poloniex.com/public?' + urlencode(params)
        conn = http.client.HTTPSConnection(urlparse(url).netloc)
        try:
            conn.request('GET', url)
            resp = json.loads(conn.getresponse().read().decode('utf-8'))
            if not self.__check_response(resp):
                raise Exception('Unexpected response ' + str(resp))
            return resp
        except Exception:
            raise
        finally:
            conn.close()    
            
    def __check_response(self, resp):
        """
        Проверка, что в ответе от биржи действительно массив свеч, а не, например, объект ошибки
        :param resp: ответ от биржи (response, преобразованный в json)
        """
        if not isinstance(resp, list):
            return False
        if len(resp) > 0:
            bar = resp[0]
            if (not 'date' in bar or 
                not 'open' in bar or 
                not 'high' in bar or 
                not 'low' in bar or
                not 'close' in bar 
                or not 'volume' in bar):
                return False
        return True
    
    def __restruture_row(self, row):
        """
        Преобразование одной строки ответа от биржи в требуемый нам формат объекта
        :param row: объект одной свечи в ответе
        """
        return {
            'timestamp': row['date'],
            'open': row['open'],
            'high': row['high'],
            'low': row['low'],
            'close': row['close'],
            'vol': row['volume'],
        }
    
    def load_bars(self, start, end = datetime.now()):
        """
        Загрузить данные по свечам и преобразовать в нужный нам формат 
        :param start: начало периода (datetime)
        :param end: конец периода (datetime) - не включается в запрос
        """
        end = end - timedelta(seconds=1)
        start_time = int(time.mktime(start.timetuple()))
        end_time = int(time.mktime(end.timetuple()))
        attempt = 1
        # Сделаем максимум пять попыток запроса на биржу Poloniex с интервалом в пять секунд, 
        # после чего дождемся следующего периода для запроса
        while(attempt <= 5):
            try:
                bars = self.__request_bars(start_time, end_time)
                return [self.__restruture_row(row) for row in bars]
            except Exception as ex:
                Utils.log('Exception while requesting bars: ', ex)
                attempt += 1
                if attempt > 5:
                    return None
                Utils.log('Will try again in 5 seconds...')
                time.sleep(5)
        

In [511]:
class Storage(): 
    "Класс для сохранения данных в файлах csv"
    
    def __init__(self, currency_pair, path='', folder='data'):
        """
        :param currency_pair: пара валют в виде BTC_ETH, потребуется для конструирования имени файла csv
        :param path: путь к директории, где будет создана папка data со всеми фалами csv (по умолчанию текущая директория проекта)
        :param folder: наименование папки со всеми файлами csv (по умолчанию data)
        """
        self.currency_pair = currency_pair
        self.folder = folder
        self.path = Utils.format_path(path)
        if not folder == '':
            self.__create_folder()
            self.folder = self.path + folder
        else:
            self.folder = self.path
        
    def __create_folder(self):
        if not os.path.exists(self.path + self.folder):
            os.makedirs(self.folder)
        
    def __write_to_file(self, file, bars):
        """
        Запись данных по свечам в файл csv
        :param file: наименование файла
        :param bars: массив свеч
        """    
        mode = 'a' if os.path.isfile(file) else 'w'
        with open(file, mode) as csvfile:
            fieldnames = ['timestamp', 'open', 'high', 'low', 'close', 'vol']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            if mode == 'w':
                writer.writeheader()
            for row in bars:
                writer.writerow(row)
    
    def filename(self, date):
        """
        Сконструировать имя файла по дате (в виде 20180328_btceth)
        :param date: дата (datetime)
        """    
        formatted_currency_pair = self.currency_pair.replace('_', '').lower()
        return '{0:%Y}{0:%m}{0:%d}_{1}'.format(date, formatted_currency_pair)
        
    def save_bars(self, bars, date = datetime.now()):
        """
        Сохранить данные по свечам за определенную дату
        :param bars: массив свеч (элемент массива - объект вида {timestamp, open, high, low, close, vol})
        :param date: дата (по умолчанию - сегодня)
        """    
        file = self.folder + '/' + self.filename(date)
        self.__write_to_file(file, bars)

In [512]:
class TimeMarksManager():
    """
    Класс для сохранения меток времени в специальном файле. 
    Нужен, чтобы запоминать, за какие периоды уже были запрошены данные, между запусками скрипта."
    """
    
    timemarks_file = 'timemarks.txt'
    format_template = '%d.%m.%Y %H:%M:%S'
    
    def __init__(self, path = ''):
        ":param path: путь к директории, где будет создан файл timemarks.txt"
        self.path = Utils.format_path(path)
    
    def save_timemark(self, date):
        """
        Сохранить метку времени
        :param date: время (datetime)
        """
        with open(self.path + self.timemarks_file, 'a') as file:
            file.write(',' + date.strftime(self.format_template))
    
    def last_timemark(self):
        "Получить последнюю метку времени"
        filepath = self.path + self.timemarks_file
        if not os.path.isfile(filepath):
            return None
        with open(filepath, 'r') as file:
            marks = file.read()
            if marks == '':
                return None
            marks = marks.split(',')
            if len(marks) > 0:
                return datetime.strptime(marks[len(marks) - 1], self.format_template)
            else:
                return None


In [513]:
class Utils():
    @staticmethod
    def round_date(date, period):
        """
        Округлить дату до кратной некоторому периоду (например, время 23:12:02 округлить до периода 300 секунд -> 23:10:00)
        :param date: дата (datetime)
        :param period: период (в секундах)
        """
        t = int(time.mktime(date.timetuple()))
        return datetime.fromtimestamp(t - t % period)
    
    @staticmethod
    def format_path(path):
        """
        Добавить / в конец пути, если его там нет
        :param path: путь
        """
        if len(path) == 0:
            return path
        if not path[len(path) - 1] == '/':
            path = path + '/'
        return path
    
    @staticmethod
    def log(*args):
        "Функция логирования"
        s = reduce((lambda x, y: str(x) + ' ' + str(y)), args)
        # Логируем в консоль, где запущен скрипт
        sys.__stdout__.write(s + '\n')

In [514]:
class BarsSaver():
    """
    Класс для сохранения в локальных файлах свечей с биржи Poloniex.
    Каждые period секунд качаем очередную свечу с периодом period. Если при запросе происходит ошибка, пробуем еще четыре раза
    с интервалом 5 секунд, после чего сдаемся и ждем следующего периода. При успешном запросе сохраним очередную свечу в файл типа
    20180715_btceth и также сохраним в файл timemarks.txt очередную отметку времени, равную концу запрошенного периода. 
    В следующий раз эта отметка времени станет началом запрашиваемого периода.
    """
    
    def __init__(self, polo, storage, beginning_of_time = datetime(2018, 1, 1)):
        """
        :param polo: экземпляр объекта Poloniex для запросов на биржу
        :param storage: экземпляр объекта Storage для сохранения данных
        :param beginning_of_time: дата, начиная с которой требуются данные (datetime)
        """
        self.polo = polo
        self.storage = storage
        self.timeMarksManager = TimeMarksManager()
        self.beginning_of_time = beginning_of_time
    
    def check(self):
        """
        Проверим, что подошло время запрашивать новую порцию данных с биржи, запросим и сохраним локально
        """
        start = self.timeMarksManager.last_timemark()
        if not start:
            start = self.beginning_of_time
        Utils.log('start of period: ', start)
        end = Utils.round_date(datetime.now(), self.polo.period)
        Utils.log('end of period: ', end)
        diff = (end - start).seconds
        Utils.log('checking if it is time for next request...')
        if (diff >= polo.period):
            Utils.log('requesting next set of bars...')
            bars = polo.load_bars(start, end)
            if bars:
                Utils.log('count of bars: ', len(bars))
                Utils.log('saving bars...')
                try:
                    storage.save_bars(bars, end)
                    self.timeMarksManager.save_timemark(end)
                except Exception as ex:
                    Utils.log('Error while saving bars locally: ', ex)
                    # Тут, конечно, возможна ситуация, когда свечи сохранились успешно, а запись метки времени в timemarks.txt 
                    # завершилась ошибкой, в результате при следующем запросе последняя свеча задублируется в файле.
                    # Тут бы предусмотреть некоторую поддержку транзакционности, но пожалуй оставлю пока как есть.
            else:
                Utils.log('Can\'t get bars for this period due to unexpected errors. See logs for more information.')
    
    def run(self):
        """
        Запуск цикла периодической закачки и сохранения данных с биржи
        """
        while True:
            self.check()
            Utils.log('going to sleep for ' + str(polo.period) + ' seconds...')
            time.sleep(polo.period)
            Utils.log('awake!')


In [497]:
currency_pair = 'BTC_ETH'
polo = Poloniex(currency_pair, period = 300)
storage = Storage(currency_pair)
barsSaver = BarsSaver(polo, storage)

barsSaver.run()


start of period:  2018-01-01 00:00:00
end of period:  2018-07-29 21:05:00
checking if it is time for next request...
requesting next set of bars...
count of bars:  60445
saving bars...
go to sleep for 300 seconds...


KeyboardInterrupt: 

In [508]:
class TestStringMethods(unittest.TestCase):
    def setUp(self):
        self.storage = Storage('BTC_ETH', 'test', 'data')
        self.timeMarksManager = TimeMarksManager('test')
        
    def test_time_marks_manager(self):
        date1 = datetime(2018, 7, 15, 15, 35, 0)
        self.timeMarksManager.save_timemark(date1)
        mark = self.timeMarksManager.last_timemark()
        self.assertEqual(mark, date1)
        date2 = datetime(2018, 7, 15, 15, 40, 0)
        self.timeMarksManager.save_timemark(date2)
        mark = self.timeMarksManager.last_timemark()
        self.assertEqual(mark, date2)
        
    def test_storage_filename(self):
        self.assertEqual(self.storage.filename(datetime(2018, 8, 1, 12, 0, 0)), '20180801_btceth')
        
    def test_utils_round_date(self):
        self.assertEqual(Utils.round_date(datetime(2018, 7, 15, 13, 24, 1), 300), datetime(2018, 7, 15, 13, 20, 0))
        self.assertEqual(Utils.round_date(datetime(2018, 7, 15, 0, 0, 0), 300), datetime(2018, 7, 15, 0, 0, 0))

    def test_utils_format_path(self):
        self.assertEqual(Utils.format_path(''), '')
        self.assertEqual(Utils.format_path('path'), 'path/')
        self.assertEqual(Utils.format_path('some/path/'), 'some/path/')

    def tearDown(self):
        if os.path.isfile('test/timemarks.txt'):
            os.remove('test/timemarks.txt')
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK
